In [0]:
import pandas as pd
import requests
import os
import zipfile 
import datetime 

In [0]:
#Account Storage data
storage_account_name = os.getenv('STORAGE_ACCOUNT_NAME_ENEM')
storage_account_key = os.getenv('STORAGE_ACCOUNT_KEY_ENEM')

#Configuring spark for consult data on Blob Storage
spark.conf.set(f"fs.azure.account.key.{storage_account_key}.blob.core.windows.net", storage_account_key)

#Layer Bronze
container_bronze= "bronze"
files_layer_bronze = dbutils.fs.ls(f"wasbs://{container_bronze}@{storage_account_name}.blob.core.windows.net/")

In [0]:
dbutils.fs.mount( 
   source=f"wasbs://{container_bronze}@{storage_account_name}.blob.core.windows.net/",
   mount_point="/mnt/enem-bronze",
   extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
)

Out[3]: True

In [0]:
def extract_source_file(source_url, save_path, chunk_size=1000):
    r = requests.get(source_url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [0]:
dbutils.fs.ls('/mnt/enem-bronze/')

Out[4]: [FileInfo(path='dbfs:/mnt/enem-bronze/MICRODADOS_ENEM_2019.csv', name='MICRODADOS_ENEM_2019.csv', size=3198954759)]

In [0]:
url = 'https://download.inep.gov.br/microdados/microdados_enem_2019.zip'
directory = '/dbfs/mnt/enem-bronze'
file_extension = '.{}'.format(url.split('/')[4].split('.')[1])
path = '{}/{}'.format(directory,url.split('/')[4])
extract_enade_source_file(url, path)

In [0]:
dbutils.fs.refreshMounts()

Mounts successfully refreshed.
Out[53]: True

In [0]:
for item in os.listdir(directory):
  if item.endswith(file_extension):
    zip_ref = zipfile.ZipFile(path)
    zip_ref.extractall(directory)
    zip_ref.close()

In [0]:
dbutils.fs.refreshMounts()